## Introduction
In this tutorial, we will be analyzing how our FEA code will deal with large deformations along with differences between p & h refinement. We will take the example that we used in the analytical performance tutorial with the stoplights and the heavy birds, and analyze it using both types of refinements.

In [1]:
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis import solver_demo_helper_functions as solver_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import numpy as np

def define_sample_problem_geom(ele_type, nx, ny, L, H):
    coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
    return coords, connect

def define_sample_problem_info(ele_type, coords, connect, L, H):
    # Identify boundaries
    boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0.0, L, 0.0, H)

    # Fixed nodes on the left edge
    fixed_nodes = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)

    # Assign distributed load on the right boundary
    q = -2.5
    dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", -50, q)

    # Assign material properties
    E = 500000.0
    nu = 0.3
    mu = E / (2.0 * (1.0 + nu))
    kappa = E / (3.0 * (1.0 - 2.0 * nu))
    
    material_props = np.array([mu, kappa])
    # Assign artificial displacement field
    displacement = np.zeros((coords.shape))
    for kk in range(0, coords.shape[0]):
        displacement[kk, 0] = coords[kk, 0] * 0.01
    return displacement, material_props, fixed_nodes, dload_info

## H-Refinement of Large Deformation

We make slight adjustments to the problem statement, modifying element properties, number of gauss points, applied load, and the type of element we use for refinement. We will first begin through h-refinement, where we utilize `D2_nn3_tri`. We see the results of this analysis in the saved gif `large_deformation_nn3.gif`. This solution is not accurate as we will see when we look into P-Refinement. 

In [2]:
ele_type = "D2_nn3_tri"
nx = 50
ny = 20
L = 2000
H = 50.0

# Generate mesh and plot it
coords, connect = define_sample_problem_geom(ele_type, nx, ny, L, H)
displacement, material_props, fixed_nodes, dload_info = define_sample_problem_info(ele_type, coords, connect, L, H)

fname = "P&H_Mesh_nn3.png"
pre_demo.plot_mesh_2D(fname, ele_type, coords, connect)

# run the example to look at the results
K, R = solver_demo.prep_for_matrix_solve(ele_type, coords.T, connect.T, material_props, displacement.T, fixed_nodes, dload_info)

nr_num_steps = 3
nr_print = True

displacements_all, nr_info_all = hyperelastic_solver(
    material_props, 
    ele_type, 
    coords.T, 
    connect.T, 
    fixed_nodes, 
    dload_info, 
    nr_print, 
    nr_num_steps, 
    nr_tol=1e-9, 
    nr_maxit=15)

fname = "large_deformation_nn3.gif"
viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)

final_disp = displacements_all[-1]  # shape: (n_nodes*ndof,)

# We need to pick a node near the end of the beam and half of the height
tip_node = 0
for i, (x, y) in enumerate(coords):
    if abs(x - L) < 1e-6 and abs(y - H/2) < H/(2*ny):
        tip_node = i
        break

tip_disp_y = final_disp[tip_node * 2 + 1]

print(f"Computed tip deflection (y): {tip_disp_y:.6f}")

# Create element quality historams
aspect_ratios, skewness, min_angles, max_angles = pre_demo.compute_element_quality_metrics(ele_type, coords, connect)
cond_nums, jac_dets = pre_demo.compute_condition_and_jacobian(ele_type, coords, connect)
plot_file = "large_deformation_nn3" + "_histograms"
pre_demo.plot_element_quality_histograms(
        fname=str(plot_file),
        super_title="Test Mesh Quality Metrics (%s)" % (ele_type),
        ele_type=ele_type,
        cond_nums=cond_nums,
        jac_dets=jac_dets,
        aspect_ratios=aspect_ratios,
        skewness=skewness,
        min_angles=min_angles,
        max_angles=max_angles
    )


Step 0, load factor = 0.333
Iteration 1, Correction=1.000000e+00, Residual=2.186928e+00, tolerance=1.000000e-09
Iteration 2, Correction=2.515533e-01, Residual=8.711405e+01, tolerance=1.000000e-09
Iteration 3, Correction=1.100804e+00, Residual=8.865407e+00, tolerance=1.000000e-09
Iteration 4, Correction=2.410569e-01, Residual=2.742946e+04, tolerance=1.000000e-09
Iteration 5, Correction=3.436761e-02, Residual=3.491813e+03, tolerance=1.000000e-09
Iteration 6, Correction=1.371190e-01, Residual=9.044665e+01, tolerance=1.000000e-09
Iteration 7, Correction=2.914355e-02, Residual=3.215578e+02, tolerance=1.000000e-09
Iteration 8, Correction=1.951878e-01, Residual=7.364351e+01, tolerance=1.000000e-09
Iteration 9, Correction=1.363305e-02, Residual=1.067321e+03, tolerance=1.000000e-09
Iteration 10, Correction=2.998342e-01, Residual=2.392057e+01, tolerance=1.000000e-09
Iteration 11, Correction=3.633523e-02, Residual=6.308795e+03, tolerance=1.000000e-09
Iteration 12, Correction=1.779335e-02, Residua

## P-Refinement of Large Deformation

Now to demonstrate the difference, when utilizing `D2_nn6_tri` element type, we witness a completely different deformation. Due to the large applied load in the X and Y directions, H refinement was not equipped to demonstrate the proper effect. With P refinement, we see the beam appropriately deform towards the bottom left and in our computed answer. Intuitively, this is confirmed by observing the saved gif `large_deformation_nn6.gif`. 

In [ ]:
ele_type = "D2_nn6_tri"
nx = 50
ny = 20
L = 2000
H = 50.0

# Generate mesh and plot it
coords, connect = define_sample_problem_geom(ele_type, nx, ny, L, H)
displacement, material_props, fixed_nodes, dload_info = define_sample_problem_info(ele_type, coords, connect, L, H)

fname = "P&H_Mesh_nn6.png"
pre_demo.plot_mesh_2D(fname, ele_type, coords, connect)

# run the example to look at the results
K, R = solver_demo.prep_for_matrix_solve(ele_type, coords.T, connect.T, material_props, displacement.T, fixed_nodes, dload_info)

nr_num_steps = 10
nr_print = True

displacements_all, nr_info_all = hyperelastic_solver(
    material_props, 
    ele_type, 
    coords.T, 
    connect.T, 
    fixed_nodes, 
    dload_info, 
    nr_print, 
    nr_num_steps, 
    nr_tol=1e-9, 
    nr_maxit=10)

fname = "large_deformation_nn6.gif"
viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)

final_disp = displacements_all[-1]  # shape: (n_nodes*ndof,)

# We need to pick a node near the end of the beam and half of the height
tip_node = 0
for i, (x, y) in enumerate(coords):
    if abs(x - L) < 1e-6 and abs(y - H/2) < H/(2*ny):
        tip_node = i
        break

tip_disp_y = final_disp[tip_node * 2 + 1]

print(f"Computed tip deflection (y): {tip_disp_y:.6f}")

# Create element quality historams
aspect_ratios, skewness, min_angles, max_angles = pre_demo.compute_element_quality_metrics(ele_type, coords, connect)
cond_nums, jac_dets = pre_demo.compute_condition_and_jacobian(ele_type, coords, connect)
plot_file = "large_deformation_nn6" + "_histograms"
pre_demo.plot_element_quality_histograms(
        fname=str(plot_file),
        super_title="Test Mesh Quality Metrics (%s)" % (ele_type),
        ele_type=ele_type,
        cond_nums=cond_nums,
        jac_dets=jac_dets,
        aspect_ratios=aspect_ratios,
        skewness=skewness,
        min_angles=min_angles,
        max_angles=max_angles
    )



Step 0, load factor = 0.100
Iteration 1, Correction=1.000000e+00, Residual=1.272428e-02, tolerance=1.000000e-09
Iteration 2, Correction=8.936032e-04, Residual=1.740875e-04, tolerance=1.000000e-09
Iteration 3, Correction=1.153207e-08, Residual=1.319848e-10, tolerance=1.000000e-09
Iteration 4, Correction=6.920354e-14, Residual=1.436630e-12, tolerance=1.000000e-09
Step 1, load factor = 0.200
Iteration 1, Correction=5.002227e-01, Residual=1.272428e-02, tolerance=1.000000e-09
Iteration 2, Correction=4.475124e-04, Residual=1.746422e-04, tolerance=1.000000e-09
Iteration 3, Correction=5.798867e-09, Residual=1.326304e-10, tolerance=1.000000e-09
Iteration 4, Correction=1.929957e-14, Residual=1.466730e-12, tolerance=1.000000e-09
Step 2, load factor = 0.300
Iteration 1, Correction=3.337294e-01, Residual=1.272428e-02, tolerance=1.000000e-09
Iteration 2, Correction=2.988164e-04, Residual=1.751991e-04, tolerance=1.000000e-09
Iteration 3, Correction=3.887949e-09, Residual=1.333252e-10, tolerance=1.000